In [1]:
from geopy.geocoders import GoogleV3
import numpy as np
import geocoder
import json
import pandas as pd
from geojson import Feature, FeatureCollection, Point
import tabula
import re

In [2]:
# Gotta figure out how to get rid of the java error.
# SO: https://stackoverflow.com/questions/54817211/java-command-is-not-found-from-this-python-process-please-ensure-java-is-inst

dsf = tabula.read_pdf('condo.pdf', pages='all',lattice=True)

In [3]:
df = dsf[0]
df.columns = df.iloc[0]
df = df.drop(df.index[0])
### This next line gets ride of the Nan ###
df = df.iloc[: , 1:]
df = df.replace(np.nan, 'Not Available', regex=True)
df = df.rename(columns={'Building / Address / City': 'building_address_city','Days on\rMarket':'days_on_market',\
                        'Price /\rSq. Ft.':'price_per_sqft'})
df['geo_address'] = df['building_address_city']
df['geo_address'] = df['geo_address'].map(lambda x: re.sub(r'\r', ' ', x))
df = df.replace('\n',' ',regex=True)
df2 = df['geo_address'].str.extract(r'(?P<building_name>\D*)(?P<address_only>\d.*)', expand=True)

Review this later (it's about how to properly merge DFs): https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html

In [4]:
result = pd.merge(df,df2,left_index=True, right_index=True)
df = result

In [5]:
%store -r google_maps_API_Key
geolocator = GoogleV3(api_key=google_maps_API_Key)

In [6]:
df['loc'] = df['address_only'].apply(geolocator.geocode, timeout=10)

In [7]:
df['loc']

1     (1425 Brickell Ave, Miami, FL 33131, USA, (25....
2     (6800 Fisher Island Dr #6821, Miami Beach, FL ...
3     (10201 Collins Ave 701 s, Bal Harbour, FL 3315...
4     (400 Alton Rd APT 3101, Miami Beach, FL 33139,...
5     (799 Crandon Blvd APT 602, Key Biscayne, FL 33...
6     (430 Grand Bay Dr APT 802, Key Biscayne, FL 33...
7     (100 South Pointe Dr APT 3102, Miami Beach, FL...
8     (5500 Island Estates Dr #601, Aventura, FL 331...
9     (Jade Ocean, 17121 Collins Ave, Sunny Isles Be...
10    (1201 20th St, Miami Beach, FL 33139, USA, (25...
Name: loc, dtype: object

In [8]:
df['point'] = df['loc'].apply(lambda loc: tuple(loc.point) if loc else None)

Columns must be same length as key explained
- https://cumsum.wordpress.com/2021/06/01/pandas-valueerror-columns-must-be-same-length-as-key/

Creating `df2` based on `df` row indicies.

https://datascience.stackexchange.com/questions/77033/creating-a-new-dataframe-with-specific-row-numbers-from-another

What's tripping this error is when the lat/lon/alititude generator is fed a Nan value to parse - it's effectively trying to split a `null` value into three separate parts - which is impossible.

The code below effectively caputres the anatomy of this error. It creates a new `df` based on a list of indicies from an existing one. Row `56` has no `point` since the `geocoder` couldn't find coordinates for it. The other three do. Splitting the `point` column fails only when row `56` - with `null` values - is attempted.
```
df2 = df.iloc[[56,57,58,59],:]
df2[['lat','lon','altitude']] = pd.DataFrame(df2['point'].to_list(),index=df2.index)
```

In [9]:
df[['lat','lon','altitude']] = pd.DataFrame(df['point'].to_list(),index=df.index)

df

,building_address_city,List Date,Close Date,days_on_market,Sale Price,Sq. Ft.,price_per_sqft,Agent,Buyer Agent,geo_address,building_name,address_only,loc,point,lat,lon,altitude
1,Four Seasons Residences\r1425 Brickell Ave PH4...,11/19/2021,3/16/2022,76,"$11,000,000.00",5798,"$1,897.21",Lourdes Alatriste,Vivian Aponte Blane,Four Seasons Residences 1425 Brickell Ave PH4B...,Four Seasons Residences,1425 Brickell Ave PH4BCD Miami,"(1425 Brickell Ave, Miami, FL 33131, USA, (25....","(25.7591323, -80.1918295, 0.0)",25.759132,-80.191829,0.0
2,Pallazzo Del Sol\r6800 Fisher Island Dr 6821\r...,12/7/2021,3/16/2022,43,"$10,900,000.00",5467,"$2,863.15",Maria Oneide Gioia,Jill Eber,Pallazzo Del Sol 6800 Fisher Island Dr 6821 Mi...,Pallazzo Del Sol,6800 Fisher Island Dr 6821 Miami Beach,"(6800 Fisher Island Dr #6821, Miami Beach, FL ...","(25.7623668, -80.14735929999999, 0.0)",25.762367,-80.147359,0.0
3,Oceana Bal Harbour Condo\r10201 Collins Ave 70...,1/14/2022,3/15/2022,14,"$9,950,000.00",3992,"$2,492.49",Nathalie Azoubel,Lindsey Lamchick,Oceana Bal Harbour Condo 10201 Collins Ave 701...,Oceana Bal Harbour Condo,10201 Collins Ave 701 S Bal Harbour,"(10201 Collins Ave 701 s, Bal Harbour, FL 3315...","(25.8951677, -80.12315, 0.0)",25.895168,-80.123150,0.0
4,Murano Grande at Portofino\r400 Alton Rd 3101\...,11/15/2021,3/15/2022,73,"$6,600,000.00",3979,"$1,658.71",Selen Arditi,Debra Samberg,Murano Grande at Portofino 400 Alton Rd 3101 M...,Murano Grande at Portofino,400 Alton Rd 3101 Miami Beach,"(400 Alton Rd APT 3101, Miami Beach, FL 33139,...","(25.7727159, -80.14041569999999, 0.0)",25.772716,-80.140416,0.0
5,Ocean Tower One Condo\r799 Crandon Blvd 602\rK...,1/21/2022,3/17/2022,39,"$5,400,000.00",3581,"$1,507.96",Nancy Batchelor PA,Lucrecia Arana Lindemann,Ocean Tower One Condo 799 Crandon Blvd 602 Key...,Ocean Tower One Condo,799 Crandon Blvd 602 Key Biscayne,"(799 Crandon Blvd APT 602, Key Biscayne, FL 33...","(25.6866419, -80.1579993, 0.0)",25.686642,-80.157999,0.0
6,Grand Bay Tower Condo\r430 Grand Bay Dr 802\rK...,1/3/2022,3/14/2022,11,"$5,100,000.00",3950,"$1,291.14",Carmencita Lopez,Dora Puig,Grand Bay Tower Condo 430 Grand Bay Dr 802 Key...,Grand Bay Tower Condo,430 Grand Bay Dr 802 Key Biscayne,"(430 Grand Bay Dr APT 802, Key Biscayne, FL 33...","(25.6930959, -80.1579994, 0.0)",25.693096,-80.157999,0.0
7,Continuum on South Beach\r100 S Pointe Dr 3102...,9/10/2021,3/15/2022,160,"$5,000,000.00",1791,"$2,791.74",Sonia Toth,Sheila Rojas,Continuum on South Beach 100 S Pointe Dr 3102 ...,Continuum on South Beach,100 S Pointe Dr 3102 Miami Beach,"(100 South Pointe Dr APT 3102, Miami Beach, FL...","(25.7676323, -80.137348, 0.0)",25.767632,-80.137348,0.0
8,Prive\r5500 Island Estates Dr 601\rAventura,4/29/2021,3/15/2022,246,"$3,900,000.00",3466,"$1,125.22",Molka Epelboim,Randi Rapp,Prive 5500 Island Estates Dr 601 Aventura,Prive,5500 Island Estates Dr 601 Aventura,"(5500 Island Estates Dr #601, Aventura, FL 331...","(25.9440574, -80.1322651, 0.0)",25.944057,-80.132265,0.0
9,Jade Ocean Condo\r17121 Collins Ave 3905/3904\...,3/2/2020,3/17/2022,699,"$3,695,000.00",4720,$782.84,Felipe Carmona,Denise Rubin,Jade Ocean Condo 17121 Collins Ave 3905/3904 S...,Jade Ocean Condo,17121 Collins Ave 3905/3904 Sunny Isles Beach,"(Jade Ocean, 17121 Collins Ave, Sunny Isles Be...","(25.9344014, -80.1210054, 0.0)",25.934401,-80.121005,0.0
10,Palau Sunset Harbour\r1201 20 St PH02\rMiami B...,11/29/2021,3/15/2022,64,"$3,500,000.00",1810,"$1,933.70",John Sandberg,Darin Tansey,Palau Sunset Harbour 1201 20 St PH02 Miami Beach,Palau Sunset Harbour,1201 20 St PH02 Miami Beach,"(1201 20th St, Miami Beach, FL 33139, USA, (25...","(25.7959607, -80.14207329999999, 0.0)",25.795961,-80.142073,0.0


Folium documentation link: https://python-visualization.github.io/folium/index.html

Folium is the library that lets us use leaflet with Python, since it's meant to be used with JavaScript.

For adding titles: https://stackoverflow.com/questions/61928013/adding-a-title-or-text-to-a-folium-map

In [10]:
#df.at[1,'Buyer Agent']=('Dora Puig, Carl Gambino')

In [11]:
df.columns

Index(['building_address_city', 'List Date', 'Close Date', 'days_on_market',
       'Sale Price', 'Sq. Ft.', 'price_per_sqft', 'Agent', 'Buyer Agent',
       'geo_address', 'building_name', 'address_only', 'loc', 'point', 'lat',
       'lon', 'altitude'],
      dtype='object')

In [14]:
import folium
from folium.plugins import MarkerCluster

m = folium.Map(location=df[["lat", "lon"]].mean().to_list(), zoom_start=11)
title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f'Recent Miami-Dade Condo Sales ')

caption_html = '''
                <p align="center" style="vertical-align: bottom; font-size:13px"><i>{}</i></p>
                '''.format('March 13th - March 19th')

marker_cluster = MarkerCluster().add_to(m)

for i,r in df.iterrows():
    location = (r["lat"], r["lon"])
    info = (r["Sale Price"], r["days_on_market"], r["Agent"], r['Buyer Agent'], r['price_per_sqft'], r['address_only'], r['building_name'])
    info = list(info)
    new_line = '<br>'
    bold_start = '<strong>'
    bold_end = '</strong>'
    text = f'Building Name: {bold_start}{info[-1]}{bold_end}{new_line} \
    Sale Price: {bold_start}{info[0]}{bold_end}{new_line}Days on Market: \
    {bold_start}{info[1]}{bold_end}{new_line}Listing Agent: {bold_start}{info[2]} \
    {bold_end}{new_line}Buying Agent: {bold_start}{info[3]}{bold_end}{new_line}Price sq ft: \
    {bold_start}{info[4]}{bold_end}{new_line}Address: {bold_start}{info[5]}{bold_end}'
    folium.Marker(location=location,
                      tooltip=text)\
    .add_to(marker_cluster)

m.get_root().html.add_child(folium.Element(title_html))
m.get_root().html.add_child(folium.Element(caption_html))

m

In [15]:
m.save('index.html')